## Calculating similarity scores to compare records

Python code that allows you to connect to the database file, send SQL queries to the database and extract the results:

In [ ]:
import pandas as pd
import sqlite3
from sqlite3 import Error

*The following `create_connection` function initializes an SQLite database connection to the specified file, handling potential errors and returning the connection object or `None`:*

In [ ]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

*The following function `run_query`, executes an SQL query using an SQLite database connection and returns the query results as a list of tuples:*

In [ ]:
def run_query(conn, query):

    cur = conn.cursor()

    cur.execute(query)

    results = cur.fetchall()

    return results

*This function, `convert_db_table_to_DF`, converts an SQLite database table into a Pandas DataFrame. Here's a brief summary:*

In [ ]:
def convert_db_table_to_DF(conn, table):

    #Step 1: We construct an SQL query (`header_query`) to fetch the column names of the table using pragma_table_info.
    header_query = "SELECT name FROM pragma_table_info('" + table + "') ORDER BY cid;"

    #Step 2: We retrieve the column names and store them in the `cols` list.
    cols_init = run_query(conn, header_query)
    cols = [cols_init[i][0] for i in range(len(cols_init))]

    #Step 3: Another SQL query (`content_query`) is constructed to fetch all the data from the specified table.
    content_query = "Select * from " + table

    #Step 4: We execute both queries using the `run_query` function and data is stored in the `df` DataFrame.
    data = run_query(conn, content_query)
    df = pd.DataFrame(data, columns = cols)
    return df

Reading the content of the customer relation (table) into Pandas DataFrame:

*The following code connects to the SQLite database we previously created named "Group_8_.db," retrieves the "customer" table from the database, and stores it as a Pandas DataFrame named `df_cust`:*

In [ ]:
database = "group8_assignment1_final_db.db"

conn = create_connection(database)
with conn:
    df_cust = convert_db_table_to_DF(conn, 'customer')
df_cust

,customer_id,street,house_number,city,country
0,123592,Hoofdstraat,17,Breda,The Netherlands
1,513914,Prinsengracht,168,Amsterdam,The Netherlands
2,200182,Diagonal,242,Barcelona,Spain
3,189204,Oudegracht,5,Utrecht,The Netherlands
4,710209,Main Street,1282,New York,USA
5,346167,Elm Street,1206,Houston,USA
6,104510,Rue Commines,15,Paris,France
7,150168,Pine Street,723,San Fransisco,USA
8,689329,Friedrichstrasse,23,Dortmund,Germany
9,200349,Cherry Road,666,Philadelphia,USA


Using a similarity function that compares two records (similar to the one in the tutorial), report the customers with similarity > 0.7.

*The following code calculates the Jaro similarity score between two strings, measuring their similarity based on common characters and their order, returning a similarity value between 0.0 and 1.0.*

In [ ]:
def jaro(s, t):
    # Step 1: We check for exact match, returns 1.0 if strings are identical
    if s == t:
        return 1.0

    # Step 2: calculate lengths of the input strings
    len_s, len_t = len(s), len(t)

    # Step 3: check for empty strings, returns 0.0 if either string is empty
    if len_s == 0 or len_t == 0:
        return 0.0

    # Step 4: We calculate maximum allowed character mismatch distance
    match_distance = max(len_s, len_t) // 2 - 1

    # We initialize match tracking lists for both strings
    s_matches, t_matches = [False] * len_s, [False] * len_t
    common_matches = 0

    # Step 5: We find common matches between the strings
    for i in range(len_s):
        start = max(0, i - match_distance)
        end = min(i + match_distance + 1, len_t)

        for j in range(start, end):
            # If a character has not been matched in the second string and matches the character in the first string, mark both characters as matched and increment common_matches
            if not t_matches[j] and s[i] == t[j]:
                s_matches[i] = t_matches[j] = True
                common_matches += 1
                break
    if common_matches == 0:
        return 0.0

    transpositions = 0
    k = 0

    # Step 6: We calculate transpositions between matched characters
    for i in range(len_s):
        if s_matches[i]:
            while not t_matches[k]:
                k += 1
            if s[i] != t[k]:
                transpositions += 1
            k += 1

    common_matches = float(common_matches)

    # Step 7: Jaro similarity score using the formula
    jaro_similarity = (common_matches / len_s + common_matches / len_t + (common_matches - transpositions) / common_matches) / 3.0

    return jaro_similarity


*Now we calculate the Jaro similarity between pairs of "street" values in a DataFrame (`df_cust`) and stores pairs with a similarity greater than 0.7 in a new DataFrame (`similar_streets_df`):*

In [ ]:
threshold = 0.7
similar_streets = []

for i in range(len(df_cust['street'])):
    for j in range(i + 1, len(df_cust['street'])):
        street1 = df_cust['street'].iloc[i]
        street2 = df_cust['street'].iloc[j]

        similarity = jaro(street1, street2)

        if similarity > threshold:
            similar_streets.append((street1, street2, similarity))

similar_streets_df = pd.DataFrame(similar_streets, columns=['street_name_1', 'street_name_2', 'similarity'])
similar_streets_df

,street_name_1,street_name_2,similarity
0,Main Street,Elm Street,0.778788
1,Main Street,Pine Street,0.878788


*Now we compute Jaro similarity scores between pairs of "country" values in a DataFrame (`df_cust`) and creates a new DataFrame (`similar_countries_df`) containing pairs with a similarity greater than 0.7:*

In [ ]:
threshold = 0.7
similar_countries = []

for i in range(len(df_cust['country'])):
    for j in range(i + 1, len(df_cust['country'])):
        country1 = df_cust['country'].iloc[i]
        country2 = df_cust['country'].iloc[j]

        similarity = jaro(country1, country2)

        if similarity > threshold:
            similar_countries.append((country1, country2, similarity))

similar_countries_df = pd.DataFrame(similar_countries, columns=['country_name_1', 'country_name_2', 'similarity'])
similar_countries_df

,country_name_1,country_name_2,similarity
0,The Netherlands,The Netherlands,1.0
1,The Netherlands,The Netherlands,1.0
2,The Netherlands,The Netherlands,1.0
3,USA,USA,1.0
4,USA,USA,1.0
5,USA,USA,1.0
6,USA,USA,1.0
7,USA,USA,1.0
8,USA,USA,1.0


*Now we calculate Jaro similarity scores between pairs of "city" values in a DataFrame (`df_cust`) and creates a new DataFrame (`similar_cities_df`) for pairs with similarity > 0.7:*

In [ ]:
threshold = 0.7
similar_cities = []

for i in range(len(df_cust['city'])):
    for j in range(i + 1, len(df_cust['city'])):
        city1 = df_cust['city'].iloc[i]
        city2 = df_cust['city'].iloc[j]

        similarity = jaro(city1, city2)

        if similarity > threshold:
            similar_streets.append((city1, city2, similarity))

similar_cities_df = pd.DataFrame(similar_streets, columns=['city_1', 'city_2', 'similarity'])
similar_cities_df

,city_1,city_2,similarity


*Given that the Jaro-Winkler similarity metric is primarily designed for comparing letters, we customized variation of it in the function "jaro_numeric" to compare the "house_numbers" and "customer_id" which contain numbers.*

*This code calculates the Jaro similarity score between two numeric values (num1 and num2), measuring their similarity based on their string representations:*

In [ ]:
def jaro_numeric(num1, num2):
    # Step 1: We convert numeric values to strings
    num1_str, num2_str = str(num1), str(num2)

    # Step 2: Calculate lengths of the string representations
    len_num1, len_num2 = len(num1_str), len(num2_str)

    # Step 3: Check for exact match, returns 1.0 if they are identical
    if num1_str == num2_str:
        return 1.0

    # Step 4: Calculate maximum allowed character mismatch distance
    match_distance = max(len_num1, len_num2) // 2 - 1
    num1_matches, num2_matches = [False] * len_num1, [False] * len_num2
    common_matches, transpositions, k = 0, 0, 0

    # Step 5: We find common matches between the strings
    for i in range(len_num1):
        start = max(0, i - match_distance)
        end = min(i + match_distance + 1, len_num2)

        for j in range(start, end):
            # If a character has not been matched in the second string and matches the character in the first string, it marks both characters as matched and increment common_matches
            if not num2_matches[j] and num1_str[i] == num2_str[j]:
                num1_matches[i] = num2_matches[j] = True
                common_matches += 1
                break

    # Step 6: If there are no common matches, returns 0.0 (no similarity)
    if common_matches == 0:
        return 0.0

    # Step 7: Calculate transpositions
    for i in range(len_num1):
        if num1_matches[i]:
            while not num2_matches[k]:
                k += 1
            if num1_str[i] != num2_str[k]:
                transpositions += 1
            k += 1
    common_matches = float(common_matches)

    # Step 8: We calculate the adapted Jaro similarity score using the formula
    similarity = (common_matches / len_num1 + common_matches / len_num2 + (common_matches - transpositions) / common_matches) / 3.0

    return similarity

*Now we calculate Jaro similarity scores between pairs of "customer_id" values in a DataFrame (`df_cust`) and creates a new DataFrame (`similar_customer_ids_df`) for pairs with a similarity greater than 0.7:*

In [ ]:
threshold = 0.7
similar_customer_ids = []

for i in range(len(df_cust['customer_id'])):
    for j in range(i + 1, len(df_cust['customer_id'])):
        customer_id1 = df_cust['customer_id'].iloc[i]
        customer_id2 = df_cust['customer_id'].iloc[j]

        similarity = jaro_numeric(customer_id1, customer_id2)

        if similarity > threshold:
            similar_customer_ids.append((customer_id1, customer_id2, similarity))

similar_customer_ids_df = pd.DataFrame(similar_customer_ids, columns=['customer_id1', 'customer_id2', 'similarity'])
similar_customer_ids_df

,customer_id1,customer_id2,similarity


*Now we compute Jaro similarity scores between pairs of "house_number" values in a DataFrame (`df_cust`) and creates a new DataFrame (`similar_house_numbers_df`) for pairs with a similarity greater than 0.7:*

In [ ]:
threshold = 0.7
similar_house_numbers = []

for i in range(len(df_cust['house_number'])):
    for j in range(i + 1, len(df_cust['house_number'])):
        house_number1 = df_cust['house_number'].iloc[i]
        house_number2 = df_cust['house_number'].iloc[j]

        similarity = jaro_numeric(house_number1, house_number2)

        if similarity > threshold:
            similar_house_numbers.append((house_number1, house_number2, similarity))

similar_house_numbers_df = pd.DataFrame(similar_house_numbers, columns=['house_number1', 'house_number2', 'similarity'])
similar_house_numbers_df

,house_number1,house_number2,similarity
0,168,1282,0.722222
1,242,1282,0.722222
